In [1]:
import numpy as np
import pandas as pd
import xlwings as xw

In [2]:
hebing = [
    '中国电子科技集团',
    '中国航天科工集团',
    '中国兵器工业集团',
    '杨茂富',
    '中国电子信息产业集团',
    '中国航天科技集团',
    '中国船舶集团',
    '江滔',
    '中国航空工业集团',
    '王柯',
    '杜春香',
    '唐富奎',
    '丁燕',
    '宏嘉科技',
    '王晓燕',
    '藏朝峰',
    '智明达',
    '康达新材',
    '周淑华',
    '王洁',
    '江苏和正',
    '楼继勇',
    '陈建新',
    '罗锋',
    '新雷能'
]

In [3]:
len(hebing)

25

In [4]:
df = pd.read_excel('../收入成本大表2020-2022.xlsx', sheet_name='2020', skiprows=1)
df.dtypes

日期        datetime64[ns]
发票号码              object
客户代码             float64
客户名称              object
工商名称              object
合并口径              object
产品代码              object
数量               float64
销售含税单价           float64
不含税销售额           float64
源单单号              object
合同号               object
出库日期      datetime64[ns]
凭证号               object
备注                object
dtype: object

In [5]:
df.rename(columns={'源单单号':'出库单号', '备注':'快递单号', '日期':'开票日期', '凭证号':'收入凭证号'}, inplace=True)

In [6]:
df['合同金额'] = df['数量']*df['销售含税单价']
df['出库数量'] = df['数量']
df['单据类型'] = '快递单'
df['运输数量'] = df['数量']
df['签收单据'] = '交付确认单'

In [7]:
df = df[['合并口径','工商名称','合同号','产品代码','数量','销售含税单价','合同金额','出库日期','出库单号','出库数量',\
    '单据类型', '运输数量','不含税销售额','收入凭证号','发票号码']]

In [8]:
df.dtypes

合并口径              object
工商名称              object
合同号               object
产品代码              object
数量               float64
销售含税单价           float64
合同金额             float64
出库日期      datetime64[ns]
出库单号              object
出库数量             float64
单据类型              object
运输数量             float64
不含税销售额           float64
收入凭证号             object
发票号码              object
dtype: object

In [9]:
df.shape

(4422, 15)

In [10]:
comp = {}  # comp[合并口径]=下属公司 1、客户年总销售量大于20万
for i in hebing:
    temp = df[df['合并口径']==i].groupby('工商名称')['不含税销售额'].agg(np.sum).sort_values()
    comp_name = temp[temp>=200000].index
    comp[i] =comp_name

In [11]:
n = 0
for i in range(len(hebing)):
    a = len(comp[hebing[i]])
    n += a    
n

55

In [12]:
contracts ={}  # contracts[公司名称]=合同编号 2、抽取金额最大的三个合同，
for i in hebing:
    for j in comp[i]:
        temp = df[(df['工商名称']==j) & (df['合并口径']==i)].groupby('合同号')['不含税销售额'].agg(np.sum).sort_values(ascending=False)
        if len(temp)<=3:
            contracts[j] = temp.index
        else:
            contracts[j] = temp.index[0:3]

In [13]:
out = pd.DataFrame()
for i in hebing: # 集团
    for j in comp[i]: # 公司
        for c_id in contracts[j]: # 合同
            temp = df[(df['合同号']==c_id) & (df['合并口径']==i) & (df['工商名称']==j)]           
            out = pd.concat([out, temp])

In [14]:
out.shape

(656, 15)

In [15]:
# out.to_excel('out.xlsx', index=False, encoding='utf-8')

In [16]:
out.dtypes

合并口径              object
工商名称              object
合同号               object
产品代码              object
数量               float64
销售含税单价           float64
合同金额             float64
出库日期      datetime64[ns]
出库单号              object
出库数量             float64
单据类型              object
运输数量             float64
不含税销售额           float64
收入凭证号             object
发票号码              object
dtype: object

In [17]:
dongfang = pd.read_excel('../东方-收入测试-2019-2021.xlsx', sheet_name='收入细节性测试2019-2020', skiprows=1)
dongfang.rename(columns={'标的物型号':'产品代码'},inplace=True)
dongfang.dtypes

收入年度                              int64
记账日期                     datetime64[ns]
收入记账凭证号                          object
客户名称/客户代码                        object
补充                               object
工商名称                             object
底稿对应情况（涉密已脱敏底稿以M后缀命名）            object
收入金额                            float64
合同号                              object
产品代码                             object
合同数量                             object
合同金额（含税）                        float64
合同金额（不含税）                       float64
出库时间                             object
出库单号                             object
发票号                              object
开票日期                     datetime64[ns]
含税金额                            float64
不含税金额                           float64
客户签收确认时间                 datetime64[ns]
客户签收人                            object
发货日期                             object
数量                               object
是否确认                             object
快递公司                             object


In [18]:
out.columns

Index(['合并口径', '工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', '出库日期', '出库单号',
       '出库数量', '单据类型', '运输数量', '不含税销售额', '收入凭证号', '发票号码'],
      dtype='object')

In [19]:
new = out.merge(dongfang, how='left', on=['工商名称','合同号', '产品代码', '数量','出库单号'])
new['签收单位'] = new['收件公司']+'/'+new['收件人']+'/'+new['客户签收人']
new['回款信息'] = new['回款凭证号']+'/'+new['对方科目']+'/'+new['票据类型']

In [20]:
new = new[['合并口径', '工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额',\
    '出库日期', '出库单号', '出库数量',\
    '快递公司', '快递单号', '发件时间','运输数量',\
    '单据类型','客户签收确认时间','签收单位',\
    '开票日期','发票号码','不含税销售额',\
    '收入记账凭证号','记账日期','收入金额',\
    '回款日期','回款信息','回款金额','票据金额',  '底稿对应情况（涉密已脱敏底稿以M后缀命名）']]

In [21]:
new.to_excel('new.xlsx',index=False, encoding='utf-8')